In [ ]:
# Usage of NVDB data
# https://www.vegvesen.no/nvdb/apidokumentasjon

In [ ]:
# Notebook config, enviroment and logging

import os
import sys
from datetime import datetime

import logging
from dotenv import load_dotenv

import pakkenellik.config as conf
from pakkenellik.log import logger
from pakkenellik.dataframe.clean_column_headers import clean_column_headers
from pakkenellik.dataframe.regions import add_municipality_name
from pakkenellik.dataframe.datetime import add_ymd, add_month_name, add_week_day, add_week_day_name

# Add module path and load config
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

# Autoreload extension
if "autoreload" not in get_ipython().extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Load the .env file into local env
load_dotenv()

# Create the config
config = conf.Config(module_path)

# Enable logging and crank up log level to DEBUG.
# This is particularly useful when developing code in your project module and using it from a notebook.
logger.setLevel(logging.INFO)

In [ ]:
# nvdbapiv3 and Geopandas
import pandas as pd
import geopandas as gpd
import nvdbapiv3

In [ ]:
ras_data = nvdbapiv3.nvdbFagdata(445)
df = pd.DataFrame(ras_data.to_records())
df = clean_column_headers(df)
df.head()

In [ ]:
df.columns

In [ ]:
df.skred_dato = pd.to_datetime(df.skred_dato, format='%Y-%m-%d')
df = add_ymd(df, "skred_dato")
df = add_month_name(df, "skred_dato")
df = add_week_day(df, "skred_dato")
df = add_week_day_name(df, "skred_dato", nynorsk=True)
df = add_municipality_name(df, "kommune")

In [ ]:
gs = gpd.GeoSeries.from_wkt(df["geometri"])
gdf = gpd.GeoDataFrame(df, geometry=gs, crs="EPSG:4326")

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d-%H%M")

# using to_file with drive=GeoJSON saves a json. This saves a geojson
with open(config.get_processed_file(f"etl_e16_{timestamp}.geojson"), 'w') as f:
    # datetime don't json, so set default str
    f.write(gdf[(gdf.vegkategori == "E") & (gdf.vegnummer == 16)][["skred_dato", "geometry"]].to_json(default=str))

In [ ]:
df.to_csv(config.get_processed_file("etl_e16.csv"), index=False)